In [52]:
import re
from bs4 import BeautifulSoup
import requests
from pathlib import Path
from bs4.element import Comment

In [67]:
admin_lte_home = Path('AdminLTE/dist/pages')

In [75]:
def extract_resource(url):
    """ """
    path = admin_lte_home.joinpath(url)
    # print(path)
    if path.exists(): 
        new_path = Path(url.replace('../../', 'static/mainframe/adminlte/'))
        new_path.parent.mkdir(exist_ok=True, parents=True)
        new_path.write_bytes(path.read_bytes())        
    else:
        new_path = Path('static/mainframe/').joinpath(re.sub('https://.*/npm/@*', '', url).replace('@', '/'))
        new_path.parent.mkdir(exist_ok=True, parents=True)
        new_path.write_bytes(requests.get(url).content)
    return new_path.as_posix()

In [76]:
def solve_resource(filename):
    with open(filename, 'r') as f: 
        soup = BeautifulSoup(f.read(), 'html.parser')    
    attrname_dict = {'link': 'href', 'script': 'src', 'img': 'src'}
    for tag in soup.find_all(['script', 'link', 'img']):
        attrname = attrname_dict.get(tag.name, None)
        if not attrname: continue
        url = tag.get(attrname)
        if not url: continue
        # print(url)
        tag[attrname] = extract_resource(url) 
    return soup.prettify()

In [77]:
original_html = solve_resource(
    admin_lte_home.joinpath('index.html').as_posix())

In [81]:
soup = BeautifulSoup(original_html, 'html.parser')

# Clear menu
sidebar_menu = soup.find(attrs={'class': 'sidebar-menu'})
li_list = []
for i, li in enumerate(sidebar_menu.find_all('li', recursive=False)):
    if i in [0, 1, 6, 7]:
        continue
    li.replace_with('')

# Clear content
soup.find(attrs={'class': 'app-content'}).replace_with('')

# Toggle switch
soup.find(attrs={'data-lte-toggle': 'sidebar'}).find('i').replace_with('🌐')

# Remove useless script
for script in soup.find_all('script'):
    src = script.attrs.get('src')
    if src is not None:
        if any([k in src for k in ['adminlte', 'bootstrap']]):
            continue    
    script.replace_with('')  

# Remove useless css
for script in soup.find_all('link'):
    href = script.attrs.get('href')
    if href is not None:
        if any([k in href for k in ['adminlte', 'bootstrap']]):
            continue    
    script.replace_with('')  

# Clear all comments
for commnet in soup.find_all(string=lambda text: isinstance(text, Comment)):
    commnet.replace_with('')

# Delete meta
for meta in soup.find_all('meta')[2:]:
    meta.replace_with('')

# Replace title
soup.find('title').string='数据中台管理'

path = Path('template/mainframe/main.html')
path.parent.mkdir(exist_ok=True, parents=True)
path.write_text(soup.prettify())

13728